In [1]:
# modules
from src.main_utils import Configuration
from src.thirdHand_data_loader import get_min_max_from_dataset

# custom visualizations
from src.motion_visualization_tools import *
from src.cvae_networks import *
from src.train_utils import *

In [2]:
project_config = Configuration()
print(project_config.data_item)

Data loaded from 11 filse, stored in a dataframe with shape (248486, 10)
Dataframe headers are: ['px', 'py', 'pz', 'v1x', 'v1y', 'v1z', 'v2x', 'v2y', 'v2z', 'hand']
Data loaded from 2 filse, stored in a dataframe with shape (54402, 10)
Dataframe headers are: ['px', 'py', 'pz', 'v1x', 'v1y', 'v1z', 'v2x', 'v2y', 'v2z', 'hand']
X_centered_scaled


In [3]:
# 3 or 5 both work!
kernel_size = 5

# between 10 and 5
first_filter_size = 9

# depth should be 2, 3, 4
depth = 2
dropout = 0.1
epochs = 10 #300

batch_size = 128
latent_dim = 8

rec_loss= "L1"
reduction= "sum" 
kld_weight = 1e-1

model_name_to_save="c_vae_model_tmp"

model = VAE_CNN(project_config.device, 
                first_filter_size, 
                kernel_size, 
                depth, 
                dropout,
                latent_dim,
                rec_loss= rec_loss,
                reduction= reduction,
                kld_weight = kld_weight)

In [4]:
train_losses ,train_rec_losses,train_kld_losses, eval_losses = train_model (model, project_config, epochs, model_name_to_save) 

Image 0 saved
0:	Total: 5748.26784	Eval loss: 7389.06128	 Rec loss: 5741.54162	 KLD loss: 6.72617	 time: 5.3s


In [5]:
test_scaling_method(project_config)    

--------------------
<class 'plotly.graph_objs._figure.Figure'> <class 'str'>


In [6]:
test_generation_method(model, project_config)    

tensor(0., device='cuda:0')


In [7]:
sample_data= None
sample_size = np.random.randint(0, 128, size= 16)

for d in project_config.train_iterator:
    x_samples = d[project_config.data_item][sample_size, :,:]
    y_sampels = d['Y'][sample_size, :,:] 
    
    z, __, __ = model.encoder(x_samples, y_sampels)

    noise = torch.normal(mean=.1, std=.21, size = z.shape).to(project_config.device)
    z = z + noise
 
    x_generated = model.decoder(z , y_sampels)
    x_rec,__, __ = model(x_samples, y_sampels)
    
    fig_1 = show_generated_motions(x_samples, "Original")
    fig_1.show()
    fig_2 = show_generated_motions(x_rec, "Reconstration")
    fig_2.show()
    fig_3 = show_generated_motions(x_generated, "Generated")
    fig_3.show()
    
    break

In [8]:
counter = 0
for d in project_config.train_iterator:
    if counter > 6:
        X = d["X"][:3, :, :]
        print(X.shape)
        min_val, max_val = get_min_max_from_dataset(X)
            
        X_scaled = torch.zeros_like(X)
        X_centered = torch.zeros_like(X)
        X_centered_scaled = torch.zeros_like(X)

        # # finding the touching point of each motion and centering the motion on that
        center_points = torch.zeros_like(X[:, 0:1, :])        
        center_points[:, 0, :3] = X[:, 9, :3]
        X_centered =  X - center_points

        # # scaling data between 0 and 1
        X_scaled =(X - min_val) / (max_val - min_val)
        
        print (torch.sum(X -(X_scaled*(max_val - min_val)+min_val)) < 0.001)
        
        # # scaling the centered data between 0 and 1
        centered_min_val, centered_max_val = get_min_max_from_dataset(X_centered)
        
        x_samples = d[project_config.data_item]
        y_samples = d["Y"]
        
        x_rec, __, __ = model(x_samples, y_samples)
        
        centered_min_val= d["centered_min_val"][0]
        centered_max_val= d["centered_max_val"][0]
        
        X_centered_scaled = (X_centered - centered_min_val) / (centered_max_val - centered_min_val)
        X_back_to_orig = (X_centered_scaled*(centered_max_val - centered_min_val)+centered_min_val)+center_points
            

        fig_01 = compare_motion_data_plots([X, X_back_to_orig], 0) 
        break
    counter +=1

torch.Size([3, 20, 9])
tensor(True, device='cuda:0')
